# SK Threshold Sweep — tutorial notebook

This notebook mirrors the simplified script that sweeps **PFA → SK thresholds** using
`pygsk.thresholds.compute_sk_thresholds`. It saves a CSV and a PNG and performs
tolerant monotonicity checks. The automatic family selector uses `mode='auto3'`.


In [ ]:
# --- Parameters (edit here) ---
M, N, d = 128, 64, 1.0
PFA_MIN, PFA_MAX, STEPS = 5e-4, 5e-2, 40
LOGSPACE = False      # set True for log-spaced PFAs
MODE = "auto3"       # or "explicit" with FAMILY
FAMILY = None         # e.g., "III" if MODE == "explicit"
FIGDIR = "_figs"
CSV_OUT = f"{FIGDIR}/threshold_sweep.csv"
PNG_OUT = f"{FIGDIR}/threshold_sweep.png"
SHOW_FIG = True
TOL = 1e-10

import os, numpy as np, csv, warnings
os.makedirs(FIGDIR, exist_ok=True)


In [ ]:
# --- Imports ---
from pygsk.thresholds import compute_sk_thresholds
import matplotlib.pyplot as plt


In [ ]:
# --- Build PFA grid ---
if LOGSPACE:
    pfas = np.logspace(np.log10(PFA_MIN), np.log10(PFA_MAX), STEPS)
else:
    pfas = np.linspace(PFA_MIN, PFA_MAX, STEPS)

lows, highs, fams, kaps = [], [], [], []
for pfa in pfas:
    if MODE == "explicit":
        lo, hi, meta = compute_sk_thresholds(M, N, d, pfa=float(pfa), mode="explicit", family=FAMILY)
    else:
        lo, hi, meta = compute_sk_thresholds(M, N, d, pfa=float(pfa), mode="auto3")
    lows.append(lo); highs.append(hi)
    if isinstance(meta, dict):
        fams.append(meta.get("family", ""))
        kaps.append(meta.get("kappa", np.nan))
    else:
        fams.append("")
        kaps.append(np.nan)

lows = np.asarray(lows, dtype=float)
highs = np.asarray(highs, dtype=float)
print(f"Computed {len(pfas)} threshold pairs.")


In [ ]:
# --- Monotonicity checks (with tolerance) ---
dl = np.diff(lows)
du = np.diff(highs)
if np.any(dl < -TOL):
    idx = np.where(dl < -TOL)[0]
    warnings.warn(
        f"lower thresholds not nondecreasing at indices {idx.tolist()} (min Δ={dl.min():.3e}). "
        "Family changes can cause tiny wiggles; consider MODE='explicit'."
    )
if np.any(du > TOL):
    idx = np.where(du > TOL)[0]
    warnings.warn(
        f"upper thresholds not nonincreasing at indices {idx.tolist()} (max Δ={du.max():.3e}). "
        "Family changes can cause tiny wiggles; consider MODE='explicit'."
    )
print("Monotonicity checks completed.")


In [ ]:
# --- Save CSV ---
with open(CSV_OUT, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["M","N","d","mode","family","pfa","lower","upper","kappa"])
    for pfa, lo, hi, fam, kap in zip(pfas, lows, highs, fams, kaps):
        w.writerow([M, N, d, MODE, FAMILY or fam, f"{pfa:.8g}", f"{lo:.16g}", f"{hi:.16g}", f"{kap:.8g}"])
print(f"Saved CSV: {CSV_OUT}")


In [ ]:
# --- Plot ---
fig, ax = plt.subplots()
ax.plot(pfas, lows,  label="lower (↑ with PFA)")
ax.plot(pfas, highs, label="upper (↓ with PFA)")
ax.plot(pfas, highs*0+1, label="expected")
# Note: linear x-axis to mirror your script; switch to log by uncommenting below
# ax.set_xscale("log")
ax.set_xlabel("PFA")
ax.set_ylabel("SK threshold")
title = f"SK thresholds vs PFA (M={M}, N={N}, d={d})"
if MODE == "explicit" and FAMILY:
    title += f"  [{FAMILY}]"
ax.set_title(title)
ax.legend()

inset = (
    f"M={M}  N={N}  d={d}\n"
    f"PFA∈[{PFA_MIN:g}, {PFA_MAX:g}]  steps={STEPS}\n"
    f"mode={MODE}" + (f"  family={FAMILY}" if FAMILY else "")
)
ax.text(0.02, 0.98, inset, transform=ax.transAxes, va="top",
        fontsize=9, bbox=dict(facecolor="white", alpha=0.85, edgecolor="none"))

fig.savefig(PNG_OUT, dpi=150, bbox_inches="tight")
print(f"Saved PNG: {PNG_OUT}")
if SHOW_FIG:
    plt.show()
